In [2]:
import numpy as np
from lib.func import fetch

In [3]:
data = '''.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\\
..../.\\\..
.-.-/..|..
.|....-|.\\
..//.|....'''.split('\n')


In [115]:
data = fetch(16).split('\n')

True

### Functions

In [33]:
def getNewCoord(beam):
    return list(np.add(beam['coord'], beam['dir']))


def buildBeam(x, y, coord):
    return { 'dir': [x, y], 'coord': [coord[0], coord[1]]  }


def isValidRange(coord, n):
    if coord[0] < 0 or coord[1] < 0:
        return False
    if coord[0] == n or coord[1] == n:
        return False
    return True


def isLit(lit, coord):
    return lit.get(coord[0]).get(coord[1]) if lit.get(coord[0]) else None


def handlePipe(beams, coord):
    if coord[1]:
        beams.append(buildBeam(1, 0, coord))
        beams.append(buildBeam(-1, 0, coord))
        return False
    return True


def handleDash(beams, coord):
    if coord[0]:
        beams.append(buildBeam(0, 1, coord))
        beams.append(buildBeam(0, -1, coord))
        return False
    return True


def handleBackSlash(beam):
    beam['dir'] = [beam['dir'][1], beam['dir'][0]]
    return True


def handleFwdSlash(beam):
    beam['dir'] = [-beam['dir'][1], -beam['dir'][0]]
    return True

def determineBeamPath(beams, beam, coord, cell, wasLit):
    if cell == '|':
        if wasLit:
            return False
        return handlePipe(beams, coord)

    if cell == '-':
        if wasLit:
            return False
        return handleDash(beams, coord)

    if cell == '\\':
        return handleBackSlash(beam)

    if cell == '/':
        return handleFwdSlash(beam)
    
    beam['coord'] = getNewCoord(beam)
    return True
    

def moveBeam(beams, beam, lit, n):
    # Get next coordinate for beam
    [x, y] = getNewCoord(beam)

    # Check for invalid range
    if not isValidRange([x, y], n):
        # print(f'  --  -- Invalid Range')
        return False
    
    # If not lit, light coord
    wasLit = isLit(lit, [x, y])
    if not wasLit:
        if not lit.get(x):
            lit[x] = {}
        lit[x][y] = True

    # determine next steps based on cell
    cell = data[x][y]
    # print(f'  -- Coord: {[x, y]}')
    # print(f'  -- Cell: {cell}')
    # print(f'  -- WasLit: {wasLit}')
    # print(f'  -- Lit: {lit}')

    return determineBeamPath(beams, beam, [x, y], cell, wasLit)


### Solution

In [34]:
def solve(data):
    n = len(data)
    # count = 0

    # Create array to hold all beams
    beams = [{ 'dir': [0, 1], 'coord': [0, 0] }]
    lit = { 0: { 0: True } }
    
    while len(beams):
        # count += 1
        # if count == 20:
        #     return lit
        # print(f'Beams: {beams}')
        # Pop beam from list
        b = beams.pop()

        valid = True
        # count2 = 0
        print(lit)
        while valid:
            # count2 += 1
            # if count2 == 30:
            #     return lit
            # print('MOVE BEAM ---')
            valid = moveBeam(beams, b, lit, n)

    return lit


solve(data)

{0: {0: True}}
{0: {0: True, 1: True}}
{0: {0: True, 1: True}}
{0: {0: True, 1: True}, 1: {1: True}, 2: {1: True}, 3: {1: True}, 4: {1: True}, 5: {1: True}, 6: {1: True}, 7: {1: True}}
{0: {0: True, 1: True}, 1: {1: True}, 2: {1: True}, 3: {1: True}, 4: {1: True}, 5: {1: True}, 6: {1: True}, 7: {1: True, 0: True}}
{0: {0: True, 1: True}, 1: {1: True}, 2: {1: True}, 3: {1: True}, 4: {1: True}, 5: {1: True}, 6: {1: True}, 7: {1: True, 0: True, 2: True, 3: True}}
{0: {0: True, 1: True}, 1: {1: True}, 2: {1: True}, 3: {1: True}, 4: {1: True}, 5: {1: True}, 6: {1: True}, 7: {1: True, 0: True, 2: True, 3: True}}


{0: {0: True, 1: True, 3: True},
 1: {1: True, 3: True},
 2: {1: True, 3: True},
 3: {1: True, 3: True},
 4: {1: True, 3: True},
 5: {1: True, 3: True},
 6: {1: True, 3: True},
 7: {1: True, 0: True, 2: True, 3: True, 4: True}}

In [105]:
test = { 'dir': [1, 0] }

In [107]:
test['dir'] = [test['dir'][1], test['dir'][0]]
test

{'dir': [0, 1]}

In [ ]:
{'0': {'0': True},
 0: {1: True},
 1: {1: True},
 2: {1: True},
 3: {1: True},
 4: {1: True},
 5: {1: True},
 6: {1: True},
 7: {1: True}}